<a href="https://colab.research.google.com/github/cse583/transformers/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Libraries/Packages

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# Initialize Model and Tokenizer

Current Model: BERT (uncased), `max_tokens=512` (can remember 512 unique words)

Tokenizer is what the transformer model uses to convert input text into tokens/embeddings which the model can understand. Tokenizer is a very important part of a transformer model and cannot be easily changed. So we are just relying on the model's own tokenizer instead of creating a custom one.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Prepare Dataset

Currently using dummy data. Need to change this to actual paths/labels later.

In [ ]:
import pandas as pd
from datasets import Dataset

## Load and Split Data

In [ ]:
# FIXME: ensure path is correct before executing this block
path_to_dataset = "paths.csv"
# Load data from csv
data = pd.read_csv(path_to_dataset)
# Ensure columns are named as 'path' and 'label'
data.columns = ["path", "count"]

In [ ]:
# Load the values of the two columns
texts = data["path"].values # series of instructions
labels = data["count"].apply(lambda x: 1 if x >= 1 else 0).values # either 0 (cold) or 1 (hot)

In [ ]:
# Split into train (0.7), validation (0.15), test sets (0.15)
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.3, random_state=42, stratify=labels
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

## Tokenize Data

In [ ]:
# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

# Training Setup

In [ ]:
# Define metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_score(labels, predictions)
    auroc = roc_auc_score(labels, logits[:, 1])
    return {"accuracy": accuracy, "auroc": auroc}

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"  # Disable W&B and only use local logging
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Training (Fine-Tuning)

This step may take a very long time.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Auroc
1,0.707300,0.695268,0.555556,0.800000
2,0.660200,0.669188,0.555556,0.600000
3,0.609200,0.662745,0.555556,0.600000


TrainOutput(global_step=9, training_loss=0.6588833199607002, metrics={'train_runtime': 51.4628, 'train_samples_per_second': 2.448, 'train_steps_per_second': 0.175, 'total_flos': 33151992975360.0, 'train_loss': 0.6588833199607002, 'epoch': 3.0})

# Evaluation

In [ ]:
# Evaluate on test dataset
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 0.6677149534225464, 'eval_accuracy': 0.6, 'eval_auroc': 0.5833333333333334, 'eval_runtime': 0.367, 'eval_samples_per_second': 27.248, 'eval_steps_per_second': 2.725, 'epoch': 3.0}


# Save Model for Future Predictions

In [ ]:
# Save model and tokenizer
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load saved model
saved_model = BertForSequenceClassification.from_pretrained("./saved_model")
saved_tokenizer = BertTokenizer.from_pretrained("./saved_model")

# Pipeline for predictions
classifier = pipeline("text-classification", model=saved_model, tokenizer=saved_tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Example prediction
new_path = "FIXME"
prediction = classifier(new_path)
print(prediction)

[{'label': 'LABEL_1', 'score': 0.6213047504425049}]
